# Generative Model Development

### Team Mandrill Rumps - Solomon Barkley, Victoria Hwang, Anna B. Stephenson

April 11, 2017

Updated April 11, 2017

The goal is to infer the volume fraction of a colloidal structural color sample from its reflectance spectrum. The generative model must be able to reproduce an experimental reflectance spectrum from a set of parameters defining the sample. The foundation of the generative model is a stochastic multiple scattering algorithm included in a separate python package at https://github.com/manoharan-lab/structural-color. Here, we describe the multiple scattering model, our generative model (and its uncertainties), and our likelihood function.

### Monte Carlo multiple scattering calculations

This model predicts the reflectance spectra of structurally colored samples of thin films made from nanoscale spheres in a disordered packing. Past models have accounted for only a single scattering event in the sample [1], but this model uses a Monte Carlo approach to account for many scattering events, leading to more accurate results. 

The Monte Carlo model simulates the trajectories of individual photons as random walkers that move through a material by sampling both the step size and the direction from distributions that are calculated using the material parameters. 

##### Material parameters

1. Particle size: the radius of the particles in the sample
2. Particle refractive index
3. Particle volume fraction 
4. Sample thickness
5. Matrix refractive index: the matrix is the material in which the particles are embedded
6. Medium refractive index: the medium is the material surrounding the the matrix/particle system, usually air

##### Procedure

1. A user-specified number of photon packets is created with an initial position, direction of propagation, and weight.
2. The photon packets travel a step size sampled from a distribution based on Beer's law [2], and the positions are updated. 
3. The photon packet weights are updated to account for the light absorbed during the step.
4. The photon packets are scattered after traveling the initial step, and new direcitons are sampled from the phase function, which is the angular distribution of the scattered intensity [3].
5. The step size is then sampled again and the process of scattering and stepping is repeated many times as specified by the user.
6. After the simulation is completed, a reflectance is calculated by counting the photon packets that exit the sample in the backscattering hemisphere. 

##### Calculating Reflectance

Each simulation sends in a number of photon packets set by the user. To calculate the reflectance, the number of photon packets that exits the sample through the backscattering hemisphere is divided by the total number of photon packets sent into the sample.

A sample consists of a film of a set thickness. The code also accounts for reflections due to the index contrast at the interface using the Fresnel equations, but only at the first air-sample interface. Refraction of the backscattered light exiting the sample is calculated using Snell's law, so that the angle of photons exiting the sample can be tracked in case the user is only interested in collecting a certain angle range of light. 

### Generative model

##### Basic generative model

The goal of the generative model is to calculate reflection spectra from structural color samples defined by a set of parameters. The Monte Carlo model described above generates theoretical spectra but these spectra do not correspond to experimental measurements. The full generative model incorporates the Monte Carlo calculations as well as additional factors to produce reflection spectra that resemble experimental data.

All of the arguments taken by the Monte Carlo model can be reliably measured for typical samples, except for particle volume fraction $\phi$, which is included in the generative model as an inference parameter. The Monte Carlo model returns a theoretical reflection calculation $R^t(\lambda, \phi)$, where $\lambda$ is the light wavelength at which reflectance is calculated. A spectrum $\left \{ R^t(\phi) \right \}$ consists of a set of $R^t_i(\lambda_i, \phi)$ values, where $\left \{ \lambda_i \right \}$ represents all wavelengths used to produce the spectrum.

The theoretical reflectance $R^t$ must be transformed into a corrected reflectance $R^c$ before it can be compared with experimental data. Frequently, not all of the incident light on a non-absorbing sample is recorded as being reflected or transmitted since some light is lost in the experimental system. Since the exact reasons for loss are not known, they cannot be accurately measured. Instead, we parameterize a constant loss level $l_0$ with a linear modulation by wavelength with strength $l_1$, so that total losses $L_i = l_0 + l_1\lambda_i$. Therefore a corrected spectrum $\left \{ R^c(\phi) \right \}$ is given by the elementwise product of $\left \{ R^c\left(\phi\right) \right \}$ and $\left \{ 1-L_i \right \}$, or:

$$
R^c_i(\lambda_i,\phi, l_0, l_1)=(1-l_0-l_1\lambda_i)R^t_i\left(\lambda_i, \phi\right)
$$

The $l_0$ and $l_1$ values are not generally known (or physically interesting), so they are marginalized over in the inference calculation (performed with MCMC so marginalization is trivial).

##### Generative model extensions

There are multiple ways to extend the basic model described above. All require modification of the underlying Monte Carlo model.

The highest priority extension is to calculate transmission spectra $\left \{ T^c(\phi) \right \}$ as well as reflection spectra. Both spectra can be measured experimentally from the same sample, so this extension would increase the predictive power of the generative model. By providing more information to inform our likelihood function, we hope to obtain a better estimate of sample volume fraction $\phi$. In general, there are some losses associated with both transmission and reflection, and there is no obvious simple relation between these values. Separate parameters describing transmission losses $l_0^*$ and $l_1^*$ will be introduced and marginalized over in the inference calculation. This addition would require large changes to the Monte Carlo code, but few changes to the generative model.

Other extensions involve increasing the amount of information that goes into the Monte Carlo calculation by allowing parameters to vary that are currently held fixed. Examples include sphere polydispersity and incident beam divergence, both of which are currently fixed at zero. Both values could be measured experimentally or sampled as parameters and marginalized over in an inference calculation. These types of extensions would help to make the generative model more physical by accounting for phenomena that are currently neglected. They would require relatively minor changes to the Monte Carlo code, but the generative model may need to be reworked for computation time concerns. The basic model includes only a single parameter ($\phi$) that affects the Monte Carlo output, but addition of more parameters would increase the dimensionality and computation time required to adequately sample parameter space.

### Model uncertainties

Due to the stochastic nature of the Monte Carlo model for scattering calculations, a calculated spectrum has uncertainties that must be included and propagated in the generative model. In general, these uncertainties will depend on the number of trajectories and scattering events we choose for a single calculation, and on the total number of runs we perform. 

The choice of the number of scattering events is determined by the "equilibration" time for the scattering calculations. This means that we need enough scattering events such that most of the photons exit the sample, which would resemble more closely the behavior of light in an experimental measurement. We decided to choose 100 scattering events for our calculations.

The number of trajectories will affect the noise level and reproducibility of the resulting spectra. The more trajectories we use, the less noisy and more reproducible the data. From previous calculations, we found that choosing 10,000 trajectories leads to very reproducible results; however, we also want to reduce the computation time of each run such that the later inference calculations are computationally accessible. Therefore, we decided to use 300 trajectories per run. The uncertainty range (set to 1 standard deviation) across the spectrum of visible wavelengths calculated from 100 of these runs is

$$
0.015 < \sigma < 0.032
$$

This range should be smaller than the measured uncertainties from the experimental data. 

### Likelihood Function

Due to computation time concerns, we hope to calculate theoretical spectra as few times as possible. Our basic likelihood function will therefore accept a pre-calculated theoretical spectrum for a given volume fraction $\left \{ R^t(\phi) \right \}$. The pre-calculated theoretical spectrum could be computed in a brute-force approach or it could perform its own random walk (in volume fraction dimension only) following a Monte Carlo procedure. There are uncertainties associated with both the stochastic Monte Carlo model and the experimental data. These two uncertainties are assumed to be Gaussian, and so their convolution is a Gaussian with variance equal to the sum of the two independent noise source variances [4]. In the approximation of a linear model, likelihood is proportional to $\exp{\left(-\chi^2/2\right)}$. The likelihood is a product over all wavelengths $\lambda_i$, and depends on the experimental reflectance $\{D_i\}$ with uncertainties $\{\sigma_{di}\}$, theoretical reflectance $\{R_i^t\}$ with uncertainties $\{\sigma_{ti}\}$, and Loss parameters $\{l_i\}$:

$$
p\left(D|M,\phi,l_0,l_1,I\right)=\left(2\pi\right)^{-N/2}\left[\prod_{i=1} ^N \left(\sigma_{di}^2+\sigma_{ti}^2\right)^{-1/2}\right]\exp{\left[\sum_{i=1}^N\frac{-\left(D_i-(1-l_0-l_1\lambda_i)R_i^t(\lambda_i,\phi)\right)^2}{2\left(\sigma_{di}^2+\sigma_{ti}^2\right)}\right]}
$$



In [1]:
def calc_likelihood(spect_theory, spect_meas, sigma_theory, sigma_meas, wavelength, l0, l1):

    """
    returns likelihood of obtaining an experimental dataset from a given spectrum
    
    Parameters:
        spect_theory: a spectrum calculated from the MC model (array of length N)
        spect_meas: dependent variable - experimental measurements (array of length N)
        sigma_theory: uncertainty associated with the probabilistic MC model (array of length N)
        sigma_meas: uncertainty assocated with the experimental measurements (array of length N)
        wavelength: independent variable (array of length N)
        l0: constant loss parameter (float)
        l1: effect of wavelength on losses (float)
    """
    
    loss = l0 + l1*wavelength
    residual = (spect_data - (1-loss)*spect_theory)
    var_eff = sigma_theory**2 + sigma_data**2
    chi_square = np.sum(residual**2/var_eff)
    prefactor = 1/np.prod(np.sqrt(2*np.pi*var_eff))
    return prefactor * np.exp(-chi_square/2)

### Application

Here, we show the code necessary to load in a simulated dataset and perform inference. The simulated data has volume fraction $\phi = 0.59$, and losses $l_0=l_1=0$. 

In [1]:
import infer_structcol as ifs
import os
directory='./infer_structcol/tests/test_data/'
ifs.convert_data([450,500,550,600,650,700,750,800], 'ref.txt','dark.txt','./infer_structcol/tests/test_data')
spect = ifs.load_spectrum('infer_structcol/tests/test_data/converted/0_data_file.txt')
samp = ifs.Sample(spect.wavelength, 0.119, 120, 1.59, 1)

ifs.sample_parameters(spect, samp, nwalkers=10, nsteps=10)



emcee: Exception while calling your likelihood function:
emcee: Exception while calling your likelihood function:
  params: [ 0.51022194  0.42908405  0.00757838]
  params: [ 0.66081461  0.41764832 -0.04219163]
  args: [   reflectance   sigma_r  wavelength
0     0.171518  0.017988       450.0
1     0.152608  0.015342       500.0
2     0.253961  0.024299       550.0
3     0.091586  0.016423       600.0
4     0.068010  0.009901       650.0
5     0.041323  0.005763       700.0
6     0.010896  0.005184       750.0
7     0.007453  0.002614       800.0, <infer_structcol.main.Sample object at 0x7f485a6bc8d0>, 0.55]  args: [   reflectance   sigma_r  wavelength
0     0.171518  0.017988       450.0
1     0.152608  0.015342       500.0
2     0.253961  0.024299       550.0
3     0.091586  0.016423       600.0
4     0.068010  0.009901       650.0
5     0.041323  0.005763       700.0
6     0.010896  0.005184       750.0
7     0.007453  0.002614       800.0, <infer_structcol.main.Sample object at 0x7f

Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/barkley/.local/lib/python3.4/site-packages/emcee/ensemble.py", line 519, in __call__
    return self.f(x, *self.args, **self.kwargs)
  File "/home/barkley/.local/lib/python3.4/site-packages/emcee/ensemble.py", line 519, in __call__
    return self.f(x, *self.args, **self.kwargs)
  File "/home/barkley/code/infer_structcol/infer_structcol/model.py", line 53, in log_posterior
    theory_spectrum = calc_reflectance(vol_frac, sample)
  File "/home/barkley/code/infer_structcol/infer_structcol/model.py", line 53, in log_posterior
    theory_spectrum = calc_reflectance(vol_frac, sample)
  File "/home/barkley/code/infer_structcol/infer_structcol/run_structcol.py", line 46, in calc_reflectance
    p, lscat, labs = mc.calc_scat(particle_size, particle_index[i], sample_index[i], volume_fraction, angles, wavelen, phase_mie=False, lscat_mie=False)
  File "/home/barkley/code/infer_structcol/infer_structcol/run_structc

emcee: Exception while calling your likelihood function:
emcee: Exception while calling your likelihood function:
  params: [ 0.601497    0.45147864  0.01299058]
  params: [ 0.58381243  0.63009835 -0.00911839]
emcee: Exception while calling your likelihood function:
emcee: Exception while calling your likelihood function:
emcee: Exception while calling your likelihood function:
  params: [ 0.56025571  0.50366186  0.07161979]
  args: [   reflectance   sigma_r  wavelength
0     0.171518  0.017988       450.0
1     0.152608  0.015342       500.0
2     0.253961  0.024299       550.0
3     0.091586  0.016423       600.0
4     0.068010  0.009901       650.0
5     0.041323  0.005763       700.0
6     0.010896  0.005184       750.0
7     0.007453  0.002614       800.0, <infer_structcol.main.Sample object at 0x7f485a6c2240>, 0.55]  params: [ 0.62179975  0.5138299  -0.00737452]
  params: [ 0.53387114  0.48372488  0.09456667]
  args: [   reflectance   sigma_r  wavelength
0     0.171518  0.017988 

  File "/home/barkley/code/structural-color-montecarlo/structcol/montecarlo.py", line 811, in calc_scat
    x = size_parameter(wavelen, n_sample, radius)


  File "/home/barkley/code/structural-color-montecarlo/structcol/montecarlo.py", line 811, in calc_scat
    x = size_parameter(wavelen, n_sample, radius)


...

  kwargs: {}
  exception:


Traceback (most recent call last):


  exception:


  File "/home/barkley/.local/lib/python3.4/site-packages/emcee/ensemble.py", line 519, in __call__
    return self.f(x, *self.args, **self.kwargs)
Traceback (most recent call last):
  File "/home/barkley/code/infer_structcol/infer_structcol/model.py", line 53, in log_posterior
    theory_spectrum = calc_reflectance(vol_frac, sample)
  File "/home/barkley/.local/lib/python3.4/site-packages/emcee/ensemble.py", line 519, in __call__
    return self.f(x, *self.args, **self.kwargs)
AttributeError: 'numpy.float64' object has no attribute 'dimensionality'
  File "/home/barkley/code/infer_structcol/infer_structcol/run_structcol.py", line 46, in calc_reflectance
    p, lscat, labs = mc.calc_scat(particle_size, particle_index[i], sample_index[i], volume_fraction, angles, wavelen, phase_mie=False, lscat_mie=False)
  File "/home/barkley/code/infer_structcol/infer_structcol/model.py", line 53, in log_posterior
    theory_spectrum = calc_reflectance(vol_frac, sample)
  File "/home/barkley/code/struc

AttributeError: 'numpy.float64' object has no attribute 'dimensionality'

### References

1. S. Magkiriadou, J. G. Park, Y. S. Kim, V. N. Manoharan (2014). “On the Absence of Red Structural Color in Photonic Glasses, Bird Feathers, and Certain Beetles.” . Phys. Rev. E, 90, 062302.
2. L. Wang, S. L. Jaques, L. Zheng (1995). MCML - Monte Carlo modeling of light transport in multi-layered tissues. Computer Methods and Programs in Biomedicine , 47, 131-146.
3. C. F. Bohren, D. R. Huffman (1983). Absorption and Scattering of Light by Small Particles. New York: Wiley.
4. P. Gregory (2010). Bayesian Logical Data Analysis for the Physical Sciences. New York: Cambridge University Press.